# Maze Search
Brief: Implements DFS/BFS/Greedy/A* search over grid mazes defined in text files.
Mazes: maze1.txt, maze2.txt, maze3.txt; `A` is start, `B` is goal, `#` are walls.
Run: from maze/, `python search.py maze1.txt` (or another file) to print solution path and explored counts.
Output: shows maze with solution marked plus search statistics. Tweak the algorithm selection in code to compare performance.

## Search utilities
High-level overview of helper classes used by different search strategies.

### Node / frontier helpers
Defines the `Node` container plus LIFO (`StackFrontier`) and FIFO (`QueueFrontier`) structures to support DFS/BFS.

In [1]:
class Node() :
    def __init__(self, state, parent, action) -> None:
        self.state = state
        self.parent = parent
        self.action = action
        #pass -> used if we want to define a function and declare it later
        
class StackFrontier() :
    def __init__(self) -> None:
        self.frontier = []
        
    def add(self, node) :
        self.frontier.append(node)
        
    def contains_state(self, state) :
        return any(state==node.state for node in self.frontier)
    
    def empty(self) :
        return len(self.frontier) == 0
    
    def remove(self) :
        if self.empty() :
            raise Exception("stack is empty")
        else :
            node = self.frontier[-1]
            self.frontier = self.frontier[ : -1]
            return node
        
class QueueFrontier(StackFrontier) :
    def remove(self) :
        if self.empty() :
            raise Exception("queue is empty")
        else :
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

### Maze class
Parses maze text files, records walls/start/goal, exposes neighbors, solves via DFS (default frontier), and renders the solution.

In [2]:
class Maze() :
    def __init__(self, filename) -> None:        
        #read file 
        with open(filename) as f :
            self.contents = f.read()
            
        if self.contents.count("A") != 1 : 
            raise Exception("there cant be more than one starting state")
        if self.contents.count("B") != 1 : 
            raise Exception("there cant be more than one goal state")
        
        #set height and width of the maze   
        self.contents = self.contents.splitlines()
        self.height = len(self.contents)
        self.width = max(len(row) for row in self.contents)
                
        #keeping track of the walls 
        self.walls = []
        for i in range(self.height) :
            row = []
            for j in range(self.width) :
                try :
                    if self.contents[i][j] == "A" :
                        self.start = (i,j)
                        row.append(False)
                    elif self.contents[i][j]=="B" :
                        self.goal = (i,j)
                        row.append(False)
                    elif self.contents[i][j] ==" " :
                        row.append(False)
                    else :
                        row.append(True)
                except IndexError :
                    row.append(False)
            self.walls.append(row)
        
        self.solution = None
        
        
    def neighbors(self, state):
        row, col = state
        candidates = [
            ("left", (row, col - 1)),
            ("right", (row, col + 1)),
            ("up", (row - 1, col)),
            ("down", (row + 1, col))
        ]

        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result
    
    def solve(self) :
        #how can i get from start(A) to goal(B)
        
        #to count the num of nodes explored
        self.num_explored = 0
        
        state = Node(state=self.start, parent=None, action=None)
        frontier = StackFrontier()
        frontier.add(state)
        
        #explored set so we dont explore things twice
        self.explored=set()
        
        #a while loop until we get a solution
        while not frontier.empty() :
        
            node = frontier.remove()
            self.num_explored +=1 
            
            if node.state == self.goal :
                #backtrack to see how we got here
                actions = []
                cells = []
                
                #we keep backtracking till we get to the start node
                while node.parent is not None :
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                #reverese everything so we can make it from start to finish not finish to start    
                actions.reverse()
                cells.reverse()
                
                #we add our solution
                self.solution = (actions, cells)
                return    
                
            #now we mark node as explored node 
            self.explored.add(node.state)
            
            for action, state in self.neighbors(node.state) :
                if state not in self.explored and not frontier.contains_state(state) :
                    child = Node(state, node, action)
                    frontier.add(child)
                    
    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i, j) == self.start:
                    print("A", end="")
                elif (i, j) == self.goal:
                    print("B", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()       

### Example run
Loads `maze2.txt`, solves it, and prints the maze with the found path.

In [3]:
m = Maze("./maze2.txt")
m.solve()
m.print()             


███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█████████████████████ █ █ █ █
█   ██********        █ █ █ █
█ █ ██*███ ██*█████████ █ █ █
█ █****█   ██B█         █ █ █
█ █*██ ████████████████ █ █ █
███*██             ████ █ █ █
███*██████████████ ██ █ █ █ █
███****         ██    █ █ █ █
██████*████████ ███████ █ █ █
██████*████             █   █
A******██████████████████████

